# Fine-tuning BERT on CSI data with MLM

In [117]:
import torch
from torch.optim import AdamW
import pandas as pd
from glob import glob
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM
from tqdm.auto import tqdm

In [2]:
# load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Downloading: 100%|██████████| 420M/420M [00:19<00:00, 22.9MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [118]:
# get path to all sub files
sub_files = glob('cleaned_subs/*.tsv')

# return all subs lines as list
def data_loader(files):
    'loads texts into a single list'
    texts = []
    for f in sub_files:
        df = pd.read_csv(f, sep='\t', usecols=['text'])
        texts += list(df["text"].values)
    return texts

data = data_loader(sub_files)
len(data)

216856

In [119]:
print(data[0])
print(data[-1])

My name is Royce Harmon.
You should watch it.


In [79]:
# tokenize the data with the BERT tokenizer
inputs = tokenizer(data, 
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors='pt')

print(inputs.keys())
print(inputs['input_ids'].shape)

In [83]:
# deep copy the inputs to labels (because we will mask the inputs)
inputs['labels'] = inputs['input_ids'].detach().clone()
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [101]:
# mask with 15% probability

# create random tensor(0-1) of size inputs
random_tensor = torch.rand(inputs['input_ids'].shape)
# create a mask with 15% selected while also ingnoring
# [cls] == 101, [sep] == 102, and [pad] == 0
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != 101)*(inputs['input_ids'] != 102)*(inputs['input_ids'] != 0)

# check sizes align
masked_tensor.shape, inputs['input_ids'].shape

(torch.Size([216856, 91]), torch.Size([216856, 91]))

In [102]:
# getting indices where masked is True
non_zero_indices = []
for mask in masked_tensor:
    non_zero_indices.append(torch.flatten(mask.nonzero()).tolist())

print(non_zero_indices[:20])

[[5], [2, 5, 13], [9], [2, 3, 9, 11], [9, 13, 16], [4], [5], [7], [1, 6], [3], [2], [6, 14], [10, 11, 13], [], [], [], [6], [7, 17, 19, 27], [2], []]


In [103]:
# replace masked values with 103
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, non_zero_indices[i]] = 103

In [105]:
## pytorch dataset class

class CSIDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        labels = self.encodings['labels'][index]
        return {
            'input_ids':input_ids, 
            'attention_mask':attention_mask,
            'token_type_ids':token_type_ids,
            'labels': labels
        }


In [106]:
# create torch dataset
dataset = CSIDataset(inputs)

In [107]:
# create torch data loader
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
    )

In [91]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [63]:
model.to(device);

In [111]:
epochs = 32
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train();


In [112]:

# training loop
for epoch in range(epochs):
    loop = tqdm(data_loader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        # print(input_ids.shape)
        labels = batch['labels']
        attention_mask =  batch['attention_mask']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch: {epoch}')
        loop.set_postfix(loss=loss.item())


Epoch: 31: 100%|██████████| 1/1 [00:05<00:00,  5.18s/it, loss=4.61]


In [114]:
model.save_pretrained('model/bert-base-uncased-csi-trained');

In [155]:
# model = BertForMaskedLM.from_pretrained("bert-base-uncased");
model.from_pretrained('model/bert-base-uncased-csi-trained');


def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = f"[CLS] {text} [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("\t[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("You should [MASK] it.", top_k=5)


	[MASK]: 'see'  | weights: 0.11975984275341034
	[MASK]: 'try'  | weights: 0.10524814575910568
	[MASK]: 'do'  | weights: 0.07006993889808655
	[MASK]: 'have'  | weights: 0.06898682564496994
	[MASK]: 'take'  | weights: 0.055882424116134644


In [157]:
import random
for sent in random.sample(data, 5):
    print(sent)
    words = sent.split(' ')
    mask_int = np.random.randint(0,len(words),1)
    words[mask_int[0]] = '[MASK]'
    masked_sent = ' '.join(words)
    print(masked_sent)
    predict_masked_sent(masked_sent, top_k=5)


Hey, I bang all my stars.
Hey, [MASK] bang all my stars.
	[MASK]: 'i'  | weights: 0.7906402945518494
	[MASK]: 'you'  | weights: 0.09978710114955902
	[MASK]: 'to'  | weights: 0.011724074371159077
	[MASK]: 'they'  | weights: 0.010610293596982956
	[MASK]: 'and'  | weights: 0.010334915481507778
I think we may have our suicide note.
I think we may have our [MASK] note.
	[MASK]: 'own'  | weights: 0.6261798739433289
	[MASK]: 'last'  | weights: 0.08776261657476425
	[MASK]: 'next'  | weights: 0.040216114372015
	[MASK]: 'final'  | weights: 0.03819900378584862
	[MASK]: 'first'  | weights: 0.024385934695601463
The killer used a plasma cutter to gain entry.
The killer used [MASK] plasma cutter to gain entry.
	[MASK]: 'a'  | weights: 0.8931524157524109
	[MASK]: 'the'  | weights: 0.07068412750959396
	[MASK]: 'his'  | weights: 0.03153619542717934
	[MASK]: 'her'  | weights: 0.0014902163529768586
	[MASK]: 'another'  | weights: 0.0007962686941027641
I'm so, so sorry to bug you, Daddy.
I'm so, so [MASK] t